# Logging using the MLFlow Logger

This notebook demonstrates how to use the logging function for MLFLow

## Installation

You can install everything from the command line using the following commands.

### Installing Anomalib

The easiest way to install anomalib is to use pip.

In [ ]:
%pip install anomalib

### Install Anomalib and MLFlow

Install anomalib with MLFlow using pip.

In [ ]:
%pip install anomalib[loggers]

### Install MLFlow

Install MLFlow using pip.

In [ ]:
%pip install -qU mlflow

# Run MLFlow Server

You can execute the following command in a seperate terminal to access the MLFlow UI.

```bash
mlflow server --backend-store-uri ./notebooks/600_loggers/mlruns/
```

Or you can return to the following cell, uncomment the cell and then execute it.

ATTENTION: This cell runs indefinitely and must be interrupted manually to continue!

In [ ]:
# !mlflow server

## Dataset Directory

This cell is to ensure we change the directory to have access to the datasets.

This part is borrowed from the datamodule mvtec notebook.

In [ ]:
from pathlib import Path

# NOTE: Provide the path to the dataset root directory.
#   If the datasets is not downloaded, it will be downloaded
#   to this directory.
dataset_root = Path.cwd().parent.parent / "datasets" / "MVTec"

## Imports

In [ ]:
import warnings

from lightning.pytorch.callbacks import EarlyStopping

from anomalib import TaskType
from anomalib.callbacks.checkpoint import ModelCheckpoint
from anomalib.data import MVTec
from anomalib.engine import Engine
from anomalib.loggers import AnomalibMLFlowLogger
from anomalib.models import Fastflow

warnings.filterwarnings("ignore")

## Data Module

Using the data module to load the MVTec dataset. But first let's print the docstring.

In [ ]:
help(MVTec)

In [ ]:
datamodule = MVTec(
    root=dataset_root,
    category="bottle",
    train_batch_size=32,
    eval_batch_size=32,
    num_workers=24,
    task=TaskType.SEGMENTATION,
)

## Model

Setup Fastflow as an example model.

In [ ]:
model = Fastflow(backbone="resnet18", flow_steps=8)

## MLFlow Logger

Setup the MLFlow logger. But first let's print the docstring.

In [ ]:
help(AnomalibMLFlowLogger)

In [ ]:
mlflow_logger = AnomalibMLFlowLogger()

## Training

### Callbacks

In [ ]:
model_checkpoint = ModelCheckpoint(mode="max", monitor="pixel_AUROC")

early_stopping = EarlyStopping(monitor="pixel_AUROC", mode="max", patience=3)

### Setup Engine

In [ ]:
callbacks = [
    model_checkpoint,
    early_stopping,
]

kwargs = {"log_every_n_steps": 3}

engine = Engine(
    callbacks=callbacks,
    accelerator="auto",
    devices=1,
    logger=mlflow_logger,  # Logger is set here
    **kwargs,
)

### Fit the Model

In [ ]:
engine.fit(model=model, datamodule=datamodule)

## Testing

In [ ]:
engine.test(model=model, dataloaders=datamodule.test_dataloader())

## Demo Track Figure

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.plot([0, 0], [2, 3])

mlflow_logger.add_image(fig, "figure_demo.png")

## Save Model to MLFlow

In [ ]:
import mlflow

with mlflow.start_run(run_id=mlflow_logger.run_id):
    mlflow.pytorch.log_model(engine.model.model, "Fastflow")

## Load Model from MLFlow

In [ ]:
model_uri = f"runs:/{mlflow_logger.run_id}/Fastflow"
mlflow.pytorch.load_model(model_uri)